In [14]:
import pandas as pd
import json
import ast
from pathlib import Path
import os

### Read data from individual csv files

In [15]:
print(os.getcwd())

c:\Users\prdie\OneDrive\Sources\sem_math_repo\classification


In [16]:
elem_set_th_df = Path("..") / "data_intermediate1" / "formula_data_elementary-set-theory.csv"
algebra_precalc_df = Path("..") / "data_intermediate1" / "formula_data_algebra-precalculus.csv"
a_geometry_df = Path("..") / "data_intermediate1" / "formula_data_analytic-geometry.csv"
elem_func_df = Path("..") / "data_intermediate1" / "formula_data_elementary-functions.csv"
elem_num_th_df = Path("..") / "data_intermediate1" / "formula_data_elementary-number-theory.csv" 
euc_geom_df = Path("..") / "data_intermediate1" / "formula_data_euclidean-geometry.csv" 
trig_df = Path("..") / "data_intermediate1" / "formula_data_trigonometry.csv"

elem_set_th_df = pd.read_csv(elem_set_th_df)
algebra_precalc_df = pd.read_csv(algebra_precalc_df)
a_geometry_df = pd.read_csv(a_geometry_df)
elem_func_df = pd.read_csv(elem_func_df)
elem_num_th_df = pd.read_csv(elem_num_th_df)
euc_geom_df = pd.read_csv(euc_geom_df)
trig_df = pd.read_csv(trig_df)

all_dfs = [elem_set_th_df, algebra_precalc_df, a_geometry_df, elem_func_df, elem_num_th_df, euc_geom_df, trig_df]
all_dfs_lengths = [d_frame.shape[0] for d_frame in all_dfs]
print(all_dfs_lengths)

[10000, 5917, 234, 182, 6253, 330, 1705]


In [17]:
trig_df.head() 

,fid,mtype,exprstr,mention,tokens,type_tokens,tags
0,6327106206a69a3f488d7164_f_4_9_0,FUNC,f,function,['f'],"['func_name', '__ANON_1']","['calculus', 'trigonometry', 'fourier-series',..."
1,6327106206a69a3f488d78b7_f_3_8_0,FUNC,\exp,the function,\exp,FUNC_NORMAL,"['trigonometry', 'complex-numbers', 'logarithms']"
2,6327106206a69a3f488d7c1a_f_2_0_0,SCAL,0,two real numbers,0,"['expr_atom', '__ANON_3']",['trigonometry']
3,6327106306a69a3f488d7e1b_f_1_2_0,FUNC,f,a function,['f'],"['func_name', '__ANON_1']","['real-analysis', 'sequences-and-series', 'tri..."
4,6327106306a69a3f488d8721_f_2_1_0,FUNC,\sin,the function name f_2_1_0,\sin,FUNC_NORMAL,"['calculus', 'integration', 'trigonometry', 'd..."


#### Merge all datasets to create a multiclass training dataset

In [18]:
def merge_dfs(df_list, intended_size_per_c):
    if intended_size_per_c == -1:
        sel_dfs = df_list
    elif intended_size_per_c > 0:
        sel_dfs = [df_list[i].sample(n=intended_size_per_c) for i in range(len(df_list))]
    else:
        print("False value")
    merged_df = pd.concat(sel_dfs, axis=0)
    return merged_df

In [19]:
intended_num_samples_per_class = -1 # set to -1 to get all available -> set arbitrary number to get more balanced dataset
all_samples_df = merge_dfs(all_dfs, intended_num_samples_per_class)
all_samples_df.head()
print("merged df size: ", all_samples_df.shape[0])


merged df size:  24621


#### Extract class labels and assign values

In [20]:
print(len(ast.literal_eval(all_samples_df.iloc[0,6])))

4


In [21]:
# analyze labels frequency and distribution
labels_ids_dict = {}
labels_count = 0
labels_frequencies_dict = {}

def cell_to_label_lists(cell_val):
    global labels_count
    labels_list = ast.literal_eval(cell_val)
    for elem in labels_list:
        if elem in labels_ids_dict.keys():
            labels_frequencies_dict[elem] += 1
        else:
            labels_frequencies_dict[elem] = 1
            labels_ids_dict[elem] = labels_count
            labels_count += 1

    return labels_list

all_samples_df["labels"] = all_samples_df["tags"].map(cell_to_label_lists)
all_samples_df.head()


,fid,mtype,exprstr,mention,tokens,type_tokens,tags,labels
0,6327106106a69a3f488d6e3e_f_8_2_0,FUNC,f,any function,['f'],"['func_name', '__ANON_1']","['elementary-set-theory', 'intuition', 'infini...","[elementary-set-theory, intuition, infinity, faq]"
1,6327106106a69a3f488d6e96_f_3_22_0,FUNC,"j: \mathbb{N} \rightarrow [0, 1]",any injection f_3_22_0,"['j', ':', '\\mathbb{N}', '\\rightarrow', '[',...","['func_def', 'func_name', '__ANON_1', 'COLON',...",['elementary-set-theory'],[elementary-set-theory]
2,6327106106a69a3f488d6e96_f_3_38_0,FUNC,b,together the identity map,['b'],"['func_name', '__ANON_1']",['elementary-set-theory'],[elementary-set-theory]
3,6327106106a69a3f488d6ef4_f_0_0_0,SET,\mathbb{Q},the set f_0_0_0,\mathbb{Q},"['set_constant', 'SET_BASIC']","['elementary-set-theory', 'irrational-numbers']","[elementary-set-theory, irrational-numbers]"
4,6327106106a69a3f488d6f48_f_0_0_0,FUNC,f,a given function,['f'],"['func_name', '__ANON_1']","['functions', 'elementary-set-theory', 'functi...","[functions, elementary-set-theory, function-an..."


In [22]:
print("Total number of classes: ", len(labels_frequencies_dict))
selected_labels = []
for k in sorted(labels_frequencies_dict, key=lambda k: labels_frequencies_dict[k], reverse=True)[:40]:
    print(str(k) + ": " + str(labels_frequencies_dict[k]) )
    selected_labels.append(k)

Total number of classes:  708
elementary-set-theory: 10215
elementary-number-theory: 6604
algebra-precalculus: 6454
functions: 3713
trigonometry: 1933
number-theory: 1368
calculus: 1080
real-analysis: 1019
discrete-mathematics: 880
combinatorics: 800
proof-verification: 758
prime-numbers: 721
cardinals: 698
proof-writing: 594
contest-math: 562
divisibility: 554
solution-verification: 522
modular-arithmetic: 522
inequality: 486
notation: 481
geometry: 425
general-topology: 396
sequences-and-series: 388
polynomials: 378
logic: 373
euclidean-geometry: 364
abstract-algebra: 335
arithmetic: 324
relations: 311
definition: 288
analytic-geometry: 276
decimal-expansion: 275
recreational-mathematics: 261
proof-explanation: 260
functional-equations: 256
elementary-functions: 251
limits: 234
induction: 230
complex-numbers: 214
terminology: 208


the top 40 classes by frequency are kept, other classes are discarded

In [23]:
# transform labels to lists of label ids

def cell_to_label_lists(cell_val):
    new_labels_ls = []
    for elem in cell_val:
        if elem in selected_labels:
            new_labels_ls.append(labels_ids_dict[elem])
    if new_labels_ls == []:
        new_labels_ls.append(10000) # 10000 is the "other" label id
    return new_labels_ls


all_samples_df = all_samples_df.drop(["fid", "exprstr", "mention", "tags"], axis=1)
all_samples_df["labels"] = all_samples_df["labels"].map(cell_to_label_lists)
all_samples_df.head()

,mtype,tokens,type_tokens,labels
0,FUNC,['f'],"['func_name', '__ANON_1']",[0]
1,FUNC,"['j', ':', '\\mathbb{N}', '\\rightarrow', '[',...","['func_def', 'func_name', '__ANON_1', 'COLON',...",[0]
2,FUNC,['b'],"['func_name', '__ANON_1']",[0]
3,SET,\mathbb{Q},"['set_constant', 'SET_BASIC']",[0]
4,FUNC,['f'],"['func_name', '__ANON_1']","[5, 0]"


In [24]:
ids_to_label_str_dict = {}
ids_to_pos_index_dict = {}
count = 0
for label in selected_labels:
    ids_to_label_str_dict[labels_ids_dict[label]] = label
    ids_to_pos_index_dict[labels_ids_dict[label]] = count 
    count += 1

In [25]:
def label_to_label_str(cell_val):
    new_list = [ids_to_label_str_dict[elem] for elem in cell_val]
    return new_list

def label_ids_to_one_hot(cell_val):
    new_list = [0 for i in range (40)]
    for label_id in cell_val:
        new_list[ids_to_pos_index_dict[label_id]] = 1
    return new_list

all_samples_df["labels_str"] = all_samples_df["labels"].map(label_to_label_str)
all_samples_df["labels"] = all_samples_df["labels"].map(label_ids_to_one_hot)
all_samples_df.head()

,mtype,tokens,type_tokens,labels,labels_str
0,FUNC,['f'],"['func_name', '__ANON_1']","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[elementary-set-theory]
1,FUNC,"['j', ':', '\\mathbb{N}', '\\rightarrow', '[',...","['func_def', 'func_name', '__ANON_1', 'COLON',...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[elementary-set-theory]
2,FUNC,['b'],"['func_name', '__ANON_1']","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[elementary-set-theory]
3,SET,\mathbb{Q},"['set_constant', 'SET_BASIC']","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[elementary-set-theory]
4,FUNC,['f'],"['func_name', '__ANON_1']","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[functions, elementary-set-theory]"


In [26]:
output_path = Path("..") / "data" / "multi_class_data_TOKENIZED_V1.csv"
all_samples_df.to_csv(output_path, index=False)
print(all_samples_df.shape[0])

24621


In [29]:
for label in selected_labels:
    one_hot_index = ids_to_pos_index_dict[labels_ids_dict[label]]
    print("label: " + str(label) + ", one hot index: " + str(one_hot_index))

SyntaxError: incomplete input (974819890.py, line 1)

#### Remove single - token formulas from dataset

In [27]:

clean_data_df = all_samples_df.loc[all_samples_df["tokens"].str.len() > 5]
clean_data_df.head()

,mtype,tokens,type_tokens,labels,labels_str
1,FUNC,"['j', ':', '\\mathbb{N}', '\\rightarrow', '[',...","['func_def', 'func_name', '__ANON_1', 'COLON',...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[elementary-set-theory]
3,SET,\mathbb{Q},"['set_constant', 'SET_BASIC']","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[elementary-set-theory]
5,FUNC,"['f', '(', 'x', ')', '=', 'x', '+', '1', '0']","['func_expr', 'func_name_arg', 'func_name', '_...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[functions, elementary-set-theory]"
6,FUNC,"['f', '(', 'x', ')', '=', '9', 'x']","['func_expr', 'func_name_arg', 'func_name', '_...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[functions, elementary-set-theory]"
7,FUNC,"['f', '(', 'x', ')', '=', 'x', '^', '2', '+', ...","['func_expr', 'func_name_arg', 'func_name', '_...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[functions, elementary-set-theory]"


In [28]:
output_path = Path("..") / "data" / "multi_class_data_TOKENIZED_V1_compact.csv"
clean_data_df.to_csv(output_path, index=False)
print(clean_data_df.shape[0])

13772
